## dataclasses

更简洁的自带写类神器：dataclasses，需要注意，版本要大于等于Python 3.7
```
def dataclass(_cls=None, *, init=True, repr=True, eq=True, order=False,
              unsafe_hash=False, frozen=False):
    """Returns the same class as was passed in, with dunder methods
    added based on the fields defined in the class.

    Examines PEP 526 __annotations__ to determine fields.

    If init is true, an __init__() method is added to the class. If
    repr is true, a __repr__() method is added. If order is true, rich
    comparison dunder methods are added. If unsafe_hash is true, a
    __hash__() method function is added. If frozen is true, fields may
    not be assigned to after instance creation.
    """

```

查看源码,其中有几个方法是默认自动生成的:
- `__init__()`:初始化对象
- `__repr__()`:将对象转化为供解释器读取的形式,返回结果为字符串
- `__eq__()`:判断两个对象的值是否相等

其他函数order,unsafe_hash,frozen默认值为False,是不会自动生成的

**当然,自定义的类继承和实现自己的方法都是可以的,和之前正常定义类一致**

In [10]:
from dataclasses import dataclass

@dataclass
class Student():
    name:str
    age:str
    school_name:str

student_1 = Student("张三", 19, '北京大学')
print(student_1.__repr__())

student_2 = Student("张三", 19, '北京大学')
print(student_1 == student_2)

Student(name='张三', age=19, school_name='北京大学')
True



### 从初始化中省略字段,即类的成员变量有默认值

目前为止我们看到的所有例子，都有一个共同特点——即我们需要为所有被声明的字段传递值，除了有默认值之外。

在那种情形下(指有默认值的情况下)，我们可以选择传递值，也可以不传递。

In [11]:
@dataclass
class Teacher():
    name:str = 'Tom'
    age:int = 30
    school_name: str = '清华大学'

teacher = Teacher()  # 不需要传参
print(teacher)

teacher_2 = Teacher(name='王五')  # 可自定义传参
teacher_3 = Teacher('马六',42,'北京大学')  # 可自定义传参
print(teacher_2)
print(teacher_3)


Teacher(name='Tom', age=30, school_name='清华大学')
Teacher(name='王五', age=30, school_name='清华大学')
Teacher(name='马六', age=42, school_name='北京大学')


### Any
如果想要开放限制,允许任意的数据类型,需要导入Any

In [12]:
from typing import Any

@dataclass
class Student():
    name: Any
    age: Any
    school_name: Any = True  # 任意值都可以    

student = Student('Bob', 18)
print(student)


Student(name='Bob', age=18, school_name=True)


### 复合初始化

你想要初始化一个变量为列表,一种简单的方式是使用__post_init__方法.

In [13]:
import random
from typing import List

def get_random_marks():
    return [random.randint(60,100) for _ in range(5)]

@dataclass
class Student:
    marks:List[int]
    
    def __post__init__(self):
        self.marks = get_random_marks()
        
student = Student(get_random_marks())
student.marks

[81, 81, 75, 100, 84]

Student类产生了一个名为 marks 的列表。我们不传递 marks 的值，而是使用`__post_init__`方法初始化

此外，我们必须在`__post_init__`里调用 get_random_marks 函数,在创建对象时还需要调用get_random_marks()。这些工作是额外的

我们可以使用 dataclasses.field 来定制化 dataclass 字段的行为以及它们在 dataclass 的影响。

In [14]:
from dataclasses import field

@dataclass
class Student:
    marks: List[int] = field(default_factory=get_random_marks)

student = Student()
student.marks

[81, 69, 89, 99, 99]

dataclasses.field 接受了一个名为 default_factory 的参数，它的作用是：如果在创建对象时没有赋值，则使用该方法初始化该字段。

default_factory 必须是一个可以调用的无参数方法(通常为一个函数)。

这样我们就可以使用复合形式初始化字段

### 实际开发可能会使用全部字段进行数据比较,dataclass又是如何提供便利方案的?

dataclass 能够自动生成< , =, >, <=和>=这些比较方法。但是这些比较方法的一个缺陷是，它们使用类中的所有字段进行比较，而这种情况往往不常见

你仅想比较用户对象的年龄、身高和体重。你不想比较姓名

In [15]:
@dataclass(order = True)
class User:
    name: str
    age: int
    height: float
    weight: float

自动生成的比较方法会比较一下的数组：

`(self.name, self.age, self.height, self.weight, )`

这将会破坏我们的意图。我们不想让姓名(name)用于比较。那么，如何使用 dataclasses.field 来实现我们的想法呢?

下面是具体步骤:

In [18]:
@dataclass(order = True)
class User:
    name: str = field(compare=False)
    age: int
    height: float
    weight: float

user_1 = User('Tom', 18, 1.75, 70)
user_2 = User('Bob', 20, 1.80, 80)

user_1 < user_2

True

默认情况下，所用的字段都用于比较，因此我们仅仅需要指定哪些字段用于比较，而实现方法是直接把不需要的字段定义为 filed(compare=False)。

一个更为简单的应用情形也可以被讨论.让我们定义一个数据类，它被用来存储一个数字激起字符串表示。我们想让比较仅仅发生在该数字的值，而不是他的字符串表示。

In [21]:
@dataclass(order=True)
class Number:
    string: str
    val: int
    
one = Number('one', 1)
nine = Number('nine', 9)

one < nine      

False

结果并不是我们想要的,限制只比较数字(和上一个案列类似)

In [22]:
@dataclass(order=True)
class Number:
    string: str = field(compare=False)
    val: int
    
one = Number('one', 1)
nine = Number('nine', 9)

one < nine      


True

对! 这才是我们想要的结果, 有时候我们想输出对面每个属性对应的值,并且要指定是哪个对象,又如何实现呢?

### 使用全部字段进行数据表示

自动生成的`__repr__`方法使用所有的字段用于表示。当然，这也不是大多数情形下的理想选择，尤其是当你的数据类有大量的字段时。单个对象的表示会变得异常臃肿，对调试来说也不利。

In [25]:
@dataclass
class User:
    name: str
    age: int
    height: float
    weight: float
    email: str
    
user_3 = User('Alan', 20, 1.70, 67, '123456@gmail.com')
user_3

User(name='Alan', age=20, height=1.7, weight=67, email='123456@gmail.com')

想象一下在你的日志里看到这样的表示吧，然后还要写一个正则表达式来搜索它。太可怕了，对吧?

当然，我们也能够个性化这种行为。考虑一个类似的使用场景，也许最合适的用于表示的属性是姓名(name),年龄(age)。那么对`__repr__`，我们仅使用它：

In [28]:
@dataclass(order=True)
class User:
    name: str = field(compare=False)
    age: int
    height: float = field(repr=False, compare=False)
    weight: float = field(repr=False, compare=False)
    email: str = field(repr=False,  compare=False)

user_4 = User('Jack', 30, 1.85, 80, '98765@gmail.com')

user_5 = User('Block', 20, 1.75, 68, '121212@gmail.com')

print(user_4 < user_5)  # Jack年长于Block,我们只关心年龄

user_4  # 输出姓名和年龄

False


User(name='Jack', age=30)

这样看起来就很棒了。调试很方便，比较也有意义!

试想一下,比如我们可能不想在初始化时设定某个字段的值。也许你在追踪一个对象的状态，并且希望它在初始化时一直被设为 False。

### 更一般地，这个值在初始化时不能够被传递

那么如何操作? default

In [33]:
@dataclass
class People:
    name: str
    job: str
    verified: bool = field(repr=False, init=False, default=False)
        # 某一类人,工作不允许校验,从事之后就不能更改

eunuch_1 = People('小古子', '太监')
print(eunuch_1)

eunuch_2 = People('小德子', '太监', True)  # 不允许修改身份
print(eunuch_2)

People(name='小古子', job='太监')


TypeError: __init__() takes 3 positional arguments but 4 were given

### 总结:
    
dataclass的用法很多,也很灵活,后续还有新知识再补充


参考文章:<https://www.baidu.com/link?url=d55eUyYTveqdoW2rrmH8uk8-fZWO7gMkm72lIDpSRgR19N_gniqvAXjfj-9HgxxiuHxL_uDlg52IG0z924ieHa&wd=&eqid=8194527400108816000000025e6a36a5>
